# Land stress impact factors

## Forestry
Exiobase categories:
- Forest area - Forestry

LC-Impact stressors
- Intensive forestry 
- Extensive forestry

In [18]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments from json file
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [20]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [21]:
# diagonalize and calculate (forestry)
diag_extension = exio3_11.satellite.diag_stressor(("Forest area - Forestry"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_extension.S is None:
    print("Calculating S")
    diag_extension.S = pymrio.calc_S(diag_extension.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_extension.D_cba, _, _, _ = pymrio.calc_accounts(diag_extension.S, exio3_11.L, Y_agg)
diag_extension.D_cba

L matrix not found, calculating it from scratch.
Calculating A
Calculating L
Calculating S


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_61798/2166560007.py:26: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                           AT        \
sector                                                   Paddy rice Wheat   
region sector                                                               
AT     Paddy rice                                               0.0   0.0   
       Wheat                                                    0.0   0.0   
       Cereal grains nec                                        0.0   0.0   
       Vegetables, fruit, nuts                                  0.0   0.0   
       Oil seeds                                                0.0   0.0   
...                                                             ...   ...   
WM     Membership organisation services n.e.c. (91)             0.0   0.0   
       Recreational, cultural and sporting services (92)        0.0   0.0   
       Other services (93)                                      0.0   0.0   
       Private households with employed persons (95)            0.0   0.0   
       Extra-territorial organizations and bodies               0.0   0.0   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                      0.0   
       Wheat                                                           0.0   
       Cereal grains nec                                               0.0   
       Vegetables, fruit, nuts                                         0.0   
       Oil seeds                                                       0.0   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)                    0.0   
       Recreational, cultural and sporting services (92)               0.0   
       Other services (93)                                             0.0   
       Private households with employed persons (95)                   0.0   
       Extra-territorial organizations and bodies                      0.0   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                            0.0   
       Wheat                                                                 0.0   
       Cereal grains nec                                                     0.0   
       Vegetables, fruit, nuts                                               0.0   
       Oil seeds                                                             0.0   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                          0.0   
       Recreational, cultural and sporting services (92)                     0.0   
       Other services (93)                                                   0.0   
       Private households with employed persons (95)                         0.0   
       Extra-territorial organizations and bodies                            0.0   

region                                                              \
sector                                                   Oil seeds   
region sector                                                        
AT     Paddy rice                                              0.0   
       Wheat                                                   0.0   
       Cereal grains nec                                       0.0   
       Vegetables, fruit, nuts                                 0.0   
       Oil seeds                                               0.0   
...                                                            ...   
WM     Membership org

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [22]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_extension.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s = pd.DataFrame(columns)
dr_s

AT        \
                                                         Paddy rice Wheat   
region sector                                                               
AT     Paddy rice                                               0.0   0.0   
       Wheat                                                    0.0   0.0   
       Cereal grains nec                                        0.0   0.0   
       Vegetables, fruit, nuts                                  0.0   0.0   
       Oil seeds                                                0.0   0.0   
...                                                             ...   ...   
WM     Membership organisation services n.e.c. (91)             0.0   0.0   
       Recreational, cultural and sporting services (92)        0.0   0.0   
       Other services (93)                                      0.0   0.0   
       Private households with employed persons (95)            0.0   0.0   
       Extra-territorial organizations and bodies               0.0   0.0   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                      0.0   
       Wheat                                                           0.0   
       Cereal grains nec                                               0.0   
       Vegetables, fruit, nuts                                         0.0   
       Oil seeds                                                       0.0   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)                    0.0   
       Recreational, cultural and sporting services (92)               0.0   
       Other services (93)                                             0.0   
       Private households with employed persons (95)                   0.0   
       Extra-territorial organizations and bodies                      0.0   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                            0.0   
       Wheat                                                                 0.0   
       Cereal grains nec                                                     0.0   
       Vegetables, fruit, nuts                                               0.0   
       Oil seeds                                                             0.0   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                          0.0   
       Recreational, cultural and sporting services (92)                     0.0   
       Other services (93)                                                   0.0   
       Private households with employed persons (95)                         0.0   
       Extra-territorial organizations and bodies                            0.0   

                                                                    \
                                                         Oil seeds   
region sector                                                        
AT     Paddy rice                                              0.0   
       Wheat                                                   0.0   
       Cereal grains nec                                       0.0   
       Vegetables, fruit, nuts                                 0.0   
       Oil seeds                                               0.0   
...                                                            ...   
WM     Membership organisation services n.e.c. (91)            0.0   
       Recreatio

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [23]:
# harmonize regions 

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

In [24]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [39]:
# load and prepare lc-impact data
# TODO: should transformation be taken into account, now only occupation is used?
lci = pd.read_excel(arguments["lc_impact_path"] + "/11-Land stress/CFs_land_Use_average.xlsx",
                sheet_name="occupation average country",
                skiprows=1,
                header=[0,1])
lci.columns = [' '.join(col).strip() for col in lci.columns]
lci.rename(columns={lci.columns[0]: "Country"}, inplace=True)
lci = lci[["Country", "Annual crops Median", "Permanent crops Median", "Pasture Median", "Extensive forestry Median", "Intensive forestry Median", "Urban Median"]]
# Calculate the mean of forest land use types 
lci["Forestry Median"] = lci[["Extensive forestry Median", "Intensive forestry Median"]].mean(axis=1)
lci = lci[["Country", "Annual crops Median", "Permanent crops Median", "Pasture Median", "Forestry Median", "Urban Median"]]

# Add country codes to all LCI datasets and remove missing ones
lci["Country_Code"] = lci["Country"].apply(get_country_code)

# Drop countries without alpha-2 code
lci.dropna(subset=["Country_Code"], inplace=True)
lci

Alpha-2 country code does not exist for  Azores
Alpha-2 country code does not exist for  Canarias
Alpha-2 country code does not exist for  Madeira
Alpha-2 country code does not exist for  Vatican City


,Country,Annual crops Median,Permanent crops Median,Pasture Median,Forestry Median,Urban Median,Country_Code
0,Afghanistan,5.566453e-16,3.447819e-16,4.042837e-16,1.001191e-16,6.337049e-16,AF
1,Albania,5.976012e-15,3.409073e-15,1.599513e-15,1.135610e-15,4.596609e-15,AL
2,Algeria,3.094926e-16,1.856075e-16,1.643208e-16,3.980529e-17,3.162822e-16,DZ
3,American Samoa,1.334275e-13,1.191927e-13,1.665278e-18,0.000000e+00,1.563116e-13,AS
4,Andorra,2.807275e-15,1.806190e-15,8.655631e-16,4.854760e-16,3.267276e-15,AD
...,...,...,...,...,...,...,...
239,Vietnam,5.988875e-15,4.463550e-15,3.379006e-15,2.445846e-15,7.183575e-15,VN
240,Wallis and Futuna,2.007363e-14,1.426736e-14,1.311224e-14,1.040864e-14,2.329513e-14,WF
241,Yemen,1.005630e-15,8.422026e-16,7.550907e-16,1.919948e-17,8.845029e-16,YE
242,Zambia,8.055880e-16,6.295117e-16,5.784345e-16,1.556082e-16,1.229837e-15,ZM


In [26]:
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

def augment_land(lci_land):
    # taiwan is missing from lc-impact, add taiwan as new row with country code TW and asia averages
    cf_annual_asia = 1.4159650959661E-15
    cf_permanent_asia = 1.02741974515257E-15
    cf_average_asia = (cf_annual_asia + cf_permanent_asia) / 2
    row = pd.DataFrame({
        "Country": ["Taiwan"],
        "Average": [cf_average_asia],
        "Annual crops Median": [cf_annual_asia],
        "Permanent crops Median": [cf_permanent_asia],
        "Pasture Median": [cf_annual_asia],
        "Country_Code": ["TW"],
    })
    lci_land = pd.concat([lci_land, row], ignore_index=True)
    return lci_land

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    print("Missing from LCI marine eutrophication:", get_missing_from_lci(exio_regions_without_row, lci))
    lci = augment_land(lci)
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions in marine eutrophication after augmentation"

Missing from LCI marine eutrophication: ['TW']


In [27]:
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Duplicates found in row regions: ['BQ', 'BQ']
Row regions: ['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 'AW', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BV', 'IO', 'VG', 'BN', 'BF', 'BI', 'KH', 'CM', 'CV', 'KY', 'CF', 'TD', 'CL', 'CX', 'CC', 'CO', 'KM', 'CG', 'CD', 'CK', 'CR', 'CI', 'CU', 'CW', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'FK', 'FO', 'FJ', 'GF', 'PF', 'TF', 'GA', 'GM', 'GE', 'GH', 'GI', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG', 'GN', 'GW', 'GY', 'HT', 'HM', 'HN', 'IS', 'IR', 'IQ', 'IM', 'IL', 'JM', 'JE', 'JO', 'KZ', 'KE', 'KI', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'LI', 'MG', 'MW', 'MY', 'MV', 'ML', 'MH', 'MQ', 'MR', 'MU', 'YT', 'FM', 'MD', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NP', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NF', 'KP', 'MP', 'OM', 'PK', 'PW', 'PS', 'PA', 'PG', 'PY', 'PE', 'PH', 'PN', 'PR', 'QA', 'RE', 'RW', 'BL', 'SH', 'KN', 'LC', 'MF', 'VC', 'WS', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC'

In [28]:
# augment dr_s to create dr_u
# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u = dr_s.copy()
dr_u = dr_u.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u = pd.concat([dr_u, combined_data])

# drop row region columns
dr_u = dr_u.drop(columns=row_regions.keys(), axis=1, level=0)
# remove row regions from consumption regions (columns)
dr_u = dr_u.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u

AT        \
                                                     Paddy rice Wheat   
AT Paddy rice                                               0.0   0.0   
   Wheat                                                    0.0   0.0   
   Cereal grains nec                                        0.0   0.0   
   Vegetables, fruit, nuts                                  0.0   0.0   
   Oil seeds                                                0.0   0.0   
...                                                         ...   ...   
ZW Membership organisation services n.e.c. (91)             0.0   0.0   
   Recreational, cultural and sporting services (92)        0.0   0.0   
   Other services (93)                                      0.0   0.0   
   Private households with employed persons (95)            0.0   0.0   
   Extra-territorial organizations and bodies               0.0   0.0   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                      0.0   
   Wheat                                                           0.0   
   Cereal grains nec                                               0.0   
   Vegetables, fruit, nuts                                         0.0   
   Oil seeds                                                       0.0   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)                    0.0   
   Recreational, cultural and sporting services (92)               0.0   
   Other services (93)                                             0.0   
   Private households with employed persons (95)                   0.0   
   Extra-territorial organizations and bodies                      0.0   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                            0.0   
   Wheat                                                                 0.0   
   Cereal grains nec                                                     0.0   
   Vegetables, fruit, nuts                                               0.0   
   Oil seeds                                                             0.0   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                          0.0   
   Recreational, cultural and sporting services (92)                     0.0   
   Other services (93)                                                   0.0   
   Private households with employed persons (95)                         0.0   
   Extra-territorial organizations and bodies                            0.0   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                              0.0   
   Wheat                                                   0.0   
   Cereal grains nec                                       0.0   
   Vegetables, fruit, nuts                                 0.0   
   Oil seeds                                               0.0   
...                                                        ...   
ZW Membership organisation services n.e.c. (91)            0.0   
   Recreational, cultural and sporting services (92)       0.0   
   Other services (93)                                     0.0   
   Private households with employed persons (95)           0.0   
   Extra-territorial organizations and bodies              0.0   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy rice                                                           0.0   
   Wheat   

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [29]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f = dr_u.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["Forest area - Forestry"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u.shape)

# multiply each column of dr_u by the respective column value 
dr_f = dr_f * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f

8800
(47600, 8800)


AT        \
                                                     Paddy rice Wheat   
AT Paddy rice                                               0.0   0.0   
   Wheat                                                    0.0   0.0   
   Cereal grains nec                                        0.0   0.0   
   Vegetables, fruit, nuts                                  0.0   0.0   
   Oil seeds                                                0.0   0.0   
...                                                         ...   ...   
ZW Membership organisation services n.e.c. (91)             0.0   0.0   
   Recreational, cultural and sporting services (92)        0.0   0.0   
   Other services (93)                                      0.0   0.0   
   Private households with employed persons (95)            0.0   0.0   
   Extra-territorial organizations and bodies               0.0   0.0   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                      0.0   
   Wheat                                                           0.0   
   Cereal grains nec                                               0.0   
   Vegetables, fruit, nuts                                         0.0   
   Oil seeds                                                       0.0   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)                    0.0   
   Recreational, cultural and sporting services (92)               0.0   
   Other services (93)                                             0.0   
   Private households with employed persons (95)                   0.0   
   Extra-territorial organizations and bodies                      0.0   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                            0.0   
   Wheat                                                                 0.0   
   Cereal grains nec                                                     0.0   
   Vegetables, fruit, nuts                                               0.0   
   Oil seeds                                                             0.0   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                          0.0   
   Recreational, cultural and sporting services (92)                     0.0   
   Other services (93)                                                   0.0   
   Private households with employed persons (95)                         0.0   
   Extra-territorial organizations and bodies                            0.0   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                              0.0   
   Wheat                                                   0.0   
   Cereal grains nec                                       0.0   
   Vegetables, fruit, nuts                                 0.0   
   Oil seeds                                               0.0   
...                                                        ...   
ZW Membership organisation services n.e.c. (91)            0.0   
   Recreational, cultural and sporting services (92)       0.0   
   Other services (93)                                     0.0   
   Private households with employed persons (95)           0.0   
   Extra-territorial organizations and bodies              0.0   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy rice                                                           0.0   
   Wheat   

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [53]:
# sort rows on lci in same order as dr_f.index.sortlevel
# Make a copy to avoid modifying the original dataframe
lci_copy = lci.copy()
lci_copy.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy = lci_copy[~lci_copy.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f.index.sortlevel
lci_reindexed = lci_copy.reindex(dr_f.index.get_level_values(0).unique())

# build array from the relevent lci stressor
# every value should be repeated 200 times (number of sectors)
cf = lci_reindexed["Forestry Median"].to_numpy()
cf = np.repeat(cf, 200) # 1D array of length 200 * number of regions in lci
# expand cf_all_effects to match the shape of dr_f
cf = np.tile(cf, (dr_f.shape[1], 1)).T

pdf = dr_f * cf
pdf_total = pdf.sum()

In [54]:
pd.DataFrame(pdf_total).to_csv("csv/pdf-land-forestry.csv", index=True)